In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold

In [5]:
df = pd.read_csv('df_pca.csv', index_col=0)
print("df_pca shape: ", df.shape)


df_x = df.drop(columns=['isFraud'])

df_pca shape:  (590540, 54)


In [6]:
df.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,PCA_V_20,PCA_V_21,PCA_V_22,PCA_V_23,PCA_V_24,PCA_V_25,PCA_V_26,PCA_V_27,PCA_V_28,PCA_V_29
0,0,86400,68.5,4,13926,-999.0,150.0,1,142.0,1,...,-0.000039,-0.000064,-0.000048,0.000023,-0.000104,0.000022,0.000031,-0.000103,0.000057,0.000292
1,0,86401,29.0,4,2755,404.0,150.0,2,102.0,1,...,0.000018,-0.000025,-0.000035,0.000087,-0.000047,0.000006,0.000045,-0.000105,-0.000006,0.000276
2,0,86469,59.0,4,4663,490.0,150.0,4,166.0,2,...,0.000018,-0.000025,-0.000035,0.000087,-0.000047,0.000006,0.000045,-0.000105,-0.000006,0.000276
3,0,86499,50.0,4,18132,567.0,150.0,2,117.0,2,...,0.000348,-0.002614,-0.004376,-0.000776,0.003337,0.001268,-0.000472,-0.001277,-0.002160,0.002301
4,0,86506,50.0,1,4497,514.0,150.0,2,102.0,1,...,0.000048,0.000033,-0.000244,-0.000005,0.000313,-0.000006,-0.000187,0.000215,0.000011,-0.000713


In [7]:
df_x.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,C1,...,PCA_V_20,PCA_V_21,PCA_V_22,PCA_V_23,PCA_V_24,PCA_V_25,PCA_V_26,PCA_V_27,PCA_V_28,PCA_V_29
0,86400,68.5,4,13926,-999.0,150.0,1,142.0,1,1.0,...,-0.000039,-0.000064,-0.000048,0.000023,-0.000104,0.000022,0.000031,-0.000103,0.000057,0.000292
1,86401,29.0,4,2755,404.0,150.0,2,102.0,1,1.0,...,0.000018,-0.000025,-0.000035,0.000087,-0.000047,0.000006,0.000045,-0.000105,-0.000006,0.000276
2,86469,59.0,4,4663,490.0,150.0,4,166.0,2,1.0,...,0.000018,-0.000025,-0.000035,0.000087,-0.000047,0.000006,0.000045,-0.000105,-0.000006,0.000276
3,86499,50.0,4,18132,567.0,150.0,2,117.0,2,2.0,...,0.000348,-0.002614,-0.004376,-0.000776,0.003337,0.001268,-0.000472,-0.001277,-0.002160,0.002301
4,86506,50.0,1,4497,514.0,150.0,2,102.0,1,1.0,...,0.000048,0.000033,-0.000244,-0.000005,0.000313,-0.000006,-0.000187,0.000215,0.000011,-0.000713


In [10]:
df['isFraud'].head()

0    0
1    0
2    0
3    0
4    0
Name: isFraud, dtype: int64

In [11]:
#Split
#split dataset
X_train, X_test, y_train, y_test = train_test_split(df_x, df.iloc[:,:1], test_size=0.3, random_state=42)

# class in train
X = X_train
y = np.array(y_train)
n_sample = y.shape[0]
n_pos_sample = y[y == 0].shape[0]
n_neg_sample = y[y == 1].shape[0]

print('numbers of sample：{}; numbers of postive class: {:.2%}; numbers of negative class: {:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))
print('feature dimension：', X.shape[1])


numbers of sample：413378; numbers of postive class: 96.54%; numbers of negative class: 3.46%
feature dimension： 53


In [12]:
print(X_train.shape, y_train.shape)

(413378, 53) (413378, 1)


In [13]:
print(y_test.shape)

(177162, 1)


In [14]:
#undersampling TomeLinks
from imblearn.under_sampling import TomekLinks

TomeLinks = TomekLinks(random_state=20)
X_train_TL, y_train_TL = TomeLinks.fit_resample(X_train, y_train.values.ravel())

Using TensorFlow backend.


KeyboardInterrupt: 

In [15]:
#random undersampling
from imblearn.under_sampling import RandomUnderSampler

RS = RandomUnderSampler(random_state=2019)
X_train_rs, y_train_rs = RS.fit_resample(X_train, y_train.values.ravel())

In [18]:
print('class after random undersampling')
n_sample = y_train_rs.shape[0]
n_pos_sample = y_train_rs[y_train_rs == 0].shape[0]
n_neg_sample = y_train_rs[y_train_rs == 1].shape[0]
print('numbers of sample：{}; isFraud=0 is {:.2%}; isFraud=1 is {:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))


class after random undersampling
numbers of sample：28644; isFraud=0 is 50.00%; isFraud=1 is 50.00%


In [29]:
#Oversampling SMOTE
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X, y = sm.fit_sample(X, y)
print('class after smote')

n_sample = y.shape[0]
n_pos_sample = y[y == 0].shape[0]
n_neg_sample = y[y == 1].shape[0]
print('numbers of sample：{}; isFraud=0 is {:.2%}; isFraud=1 is {:.2%}'.format(n_sample,
                                                   n_pos_sample / n_sample,
                                                   n_neg_sample / n_sample))
X_train = pd.DataFrame(X_train)
X_train_Smote = pd.DataFrame(X)
X_train_Smote.columns = X_train.columns
y_train_Smote = pd.DataFrame(y)


/Users/xiaoweifeng/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


class after smote
numbers of sample：798112; isFraud=0 is 50.00%; isFraud=1 is 50.00%


In [51]:
print(X_train_Smote.shape)
print(y_train_Smote.shape)

(798112, 55)
(798112, 1)


In [19]:
#Logistic before solvinged imblanced
#LogisticRegression
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
#before imputation

clf = LogisticRegression(random_state=2,solver='lbfgs',
                         multi_class='multinomial').fit(X_train, y_train.values.ravel())

y_pred_l1 = clf.predict(X_test)
print('before logistic')
print('clf_score: ',clf.score(X_test, y_test.values.ravel()))
print("Accuracy on training set: {:.3f}".format(clf.score(X_train, y_train.values.ravel())))
print("Accuracy on test set: {:.3f}".format(clf.score(X_test, y_test.values.ravel())))
print(confusion_matrix(y_test, y_pred_l1))


before logistic
clf_score:  0.9642079001140199
Accuracy on training set: 0.965
Accuracy on test set: 0.964
[[170821      0]
 [  6341      0]]


In [22]:
from sklearn.metrics import auc, roc_curve, classification_report
print('before resampling')
print(classification_report(y_test, y_pred_l1))

before resampling
              precision    recall  f1-score   support

           0       0.96      1.00      0.98    170821
           1       0.00      0.00      0.00      6341

    accuracy                           0.96    177162
   macro avg       0.48      0.50      0.49    177162
weighted avg       0.93      0.96      0.95    177162



/Users/xiaoweifeng/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiaoweifeng/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/xiaoweifeng/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [24]:
#radom sampling
clf = LogisticRegression(random_state=2,solver='lbfgs',
                         multi_class='multinomial').fit(X_train_rs, y_train_rs.ravel())

y_pred_l2 = clf.predict(X_test)
print(clf.score(X_test, y_test.values.ravel()))
print("Accuracy on training set: {:.3f}".format(clf.score(X_train_rs, y_train_rs.ravel())))
print("Accuracy on test set: {:.3f}".format(clf.score(X_test, y_test)))
print(confusion_matrix(y_test, y_pred_l2))


0.58503516555469
Accuracy on training set: 0.544
Accuracy on test set: 0.585
[[100521  70300]
 [  3216   3125]]


In [27]:
print("random undersampling")
print(classification_report(y_test, y_pred_l2))

random undersampling
              precision    recall  f1-score   support

           0       0.97      0.59      0.73    170821
           1       0.04      0.49      0.08      6341

    accuracy                           0.59    177162
   macro avg       0.51      0.54      0.41    177162
weighted avg       0.94      0.59      0.71    177162



In [16]:
#after imputation TmoteLink
clf = LogisticRegression(random_state=2, solver='lbfgs',
                         multi_class='multinomial').fit(X_train_TL, y_train_TL)

y_pred_l3 = clf.predict(X_test)



# clf.score(X_test_fill, y_test.values.ravel())
print('after logistic')
print("Accuracy on training set: {:.3f}".format(clf.score(X_train_TL, y_train_TL)))
print("Accuracy on test set: {:.3f}".format(clf.score(X_test, y_test.values)))
print(confusion_matrix(y_test, y_pred_l3))


after logistic
Accuracy on training set: 0.965
Accuracy on test set: 0.964
[[170821      0]
 [  6341      0]]


In [41]:
print(classification_report(y_test, y_pred_l3))

              precision    recall  f1-score   support

           0       0.97      0.54      0.70    170821
           1       0.04      0.50      0.07      6341

    accuracy                           0.54    177162
   macro avg       0.50      0.52      0.38    177162
weighted avg       0.93      0.54      0.67    177162



In [30]:
#after oversampling(smote)
clf = LogisticRegression(random_state=2, solver='lbfgs',
                         multi_class='multinomial').fit(X_train_Smote, y_train_Smote.values.ravel())

y_pred_l4 = clf.predict(X_test)



# clf.score(X_test_fill, y_test.values.ravel())
print('after logistic')
print("Accuracy on training set: {:.3f}".format(clf.score(X_train_Smote, y_train_Smote.values.ravel())))
print("Accuracy on test set: {:.3f}".format(clf.score(X_test, y_test.values.ravel())))
print(confusion_matrix(y_test, y_pred_l4))



after logistic
Accuracy on training set: 0.524
Accuracy on test set: 0.540
[[92508 78313]
 [ 3150  3191]]


In [31]:
print('SMOTE')
print(classification_report(y_test, y_pred_l4))

SMOTE
              precision    recall  f1-score   support

           0       0.97      0.54      0.69    170821
           1       0.04      0.50      0.07      6341

    accuracy                           0.54    177162
   macro avg       0.50      0.52      0.38    177162
weighted avg       0.93      0.54      0.67    177162



In [22]:
print(X_train.shape)
print(y_train.shape)

(413378, 55)
(413378,)


In [32]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(random_state=0, max_depth = 10, class_weight='balanced')
tree.fit(X_train, y_train)
y_pred_tree = tree.predict(X_test)
print("Accuracy on training set: {:.3f}".format(tree.score(X_train, y_train)))
print("Accuracy on test set: {:.3f}".format(tree.score(X_test, y_test)))

Accuracy on training set: 0.875
Accuracy on test set: 0.872


In [33]:
print("Decision Tree")
print(classification_report(y_test, y_pred_tree))

Decision Tree
              precision    recall  f1-score   support

           0       0.99      0.88      0.93    170821
           1       0.18      0.72      0.29      6341

    accuracy                           0.87    177162
   macro avg       0.58      0.80      0.61    177162
weighted avg       0.96      0.87      0.91    177162

